# Attempt to fine-tune GPT3 to translate between languages
The first language will be our made up language (see notes on get_bible as to why we think GPT is cheating)
The thesis is that fine-tuning will cause GPT3 to learn the fake language as a new language by assigning the embeddings of the
new words essentially to the same dimensional space as the english.  Should be trivial to translate it back as it is a word 
for word translation.



In [ ]:
# Q. what is John 3:2?


In [ ]:
from lib.config import get_config
import pandas as pd

bible = pd.read_csv('data/bible.bbe.csv')

# Assuming the gospels have a lot of overlap include the synoptic gospels plus John to train
# then predict Mark (which should be easy as Matthew and Luke may have copied from him)
train = bible[bible['book'].isin(['MAT','LUK','JHN'])]
validate = bible[bible['book'].isin(['MRK'])]

In [131]:
train

,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig
23213,MAT 1:1,MAT,1,1,"The book of the genealogy of Jesus Christ, the...","The book of the generation of Jesus Christ, th...","The book of the generation of Jesus Christ, th...","The book of the generations of Jesus Christ, t...","The book of the generacioun of Jhesu Crist, th...","Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, l..."
23214,MAT 1:2,MAT,1,2,Abraham became the father of Isaac. Isaac beca...,Abraham begat Isaac; and Isaac begat Jacob; an...,Abraham begat Isaac; and Isaac begat Jacob; an...,The son of Abraham was Isaac; and the son of I...,Abraham bigat Isaac. Isaac bigat Jacob. Jacob ...,Lxi zur us Efvexen hez Ozeek; erw lxi zur us O...
23215,MAT 1:3,MAT,1,3,Judah became the father of Perez and Zerah by ...,and Judah begat Perez and Zerah of Tamar; and ...,And Judas begat Phares and Zara of Thamar; and...,And the sons of Judah were Perez and Zerah by ...,"Judas bigat Fares and Zaram, of Tamar. Fares b...",Erw lxi zurz us Yawex hivi Qivid erw Divex fj ...
23216,MAT 1:4,MAT,1,4,Ram became the father of Amminadab. Amminadab ...,and Ram begat Amminadab; and Amminadab begat N...,And Aram begat Aminadab; and Aminadab begat Na...,And the son of Ram was Amminadab; and the son ...,Esrom bigat Aram. Aram bigat Amynadab. Amynada...,Erw lxi zur us Ven hez Ennorewef; erw lxi zur ...
23217,MAT 1:5,MAT,1,5,Salmon became the father of Boaz by Rahab. Boa...,and Salmon begat Boaz of Rahab; and Boaz begat...,And Salmon begat Booz of Rachab; and Booz bega...,And the son of Salmon by Rahab was Boaz; and t...,"Salmon bigat Booz, of Raab. Booz bigat Obeth, ...",Erw lxi zur us Zepnur fj Vexef hez Fued; erw l...
...,...,...,...,...,...,...,...,...,...,...
26987,JHN 21:21,JHN,21,21,"Peter, seeing him, said to Jesus, “Lord, what ...","Peter therefore seeing him saith to Jesus, Lor...","Peter seeing him saith to Jesus, Lord, and wha...","Seeing him, Peter said to Jesus, What about th...","Therfor whanne Petre hadde seyn this, he seith...","Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual lx..."
26988,JHN 21:22,JHN,21,22,"Jesus said to him, “If I desire that he stay u...","Jesus saith unto him, If I will that he tarry ...","Jesus saith unto him, If I will that he tarry ...","Jesus said to him, If it is my desire for him ...","Jhesus seith to him, So I wole that he dwelle ...","Yizaz zeow lu xon, Os ol oz nj wizovi suv xon ..."
26989,JHN 21:23,JHN,21,23,This saying therefore went out among the broth...,This saying therefore went forth among the bre...,Then went this saying abroad among the brethre...,So this saying went about among the brothers t...,Therfor this word wente out among the brithere...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz l...
26990,JHN 21:24,JHN,21,24,This is the disciple who testifies about these...,This is the disciple that beareth witness of t...,This is the disciple which testifieth of these...,This is the disciple who gives witness about t...,"This is thilke disciple, that berith witnessyn...",Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual l...


In [132]:
df2 = pd.DataFrame({'prompt': train['0'], 'completion': train['birrig']})
df2['completion'] = df2['completion'].apply(lambda x: " " + x + '\n###')
df2['prompt'] = df2['prompt'].apply(lambda x: x + "\n\n###\n\n")
df2.to_json('data/train_exp1.jsonl', orient='records', lines=True)
df2


,prompt,completion
23213,MAT 1:1\n\n###\n\n,"Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, ..."
23214,MAT 1:2\n\n###\n\n,Lxi zur us Efvexen hez Ozeek; erw lxi zur us ...
23215,MAT 1:3\n\n###\n\n,Erw lxi zurz us Yawex hivi Qivid erw Divex fj...
23216,MAT 1:4\n\n###\n\n,Erw lxi zur us Ven hez Ennorewef; erw lxi zur...
23217,MAT 1:5\n\n###\n\n,Erw lxi zur us Zepnur fj Vexef hez Fued; erw ...
...,...,...
26987,JHN 21:21\n\n###\n\n,"Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual l..."
26988,JHN 21:22\n\n###\n\n,"Yizaz zeow lu xon, Os ol oz nj wizovi suv xon..."
26989,JHN 21:23\n\n###\n\n,Zu lxoz zejorg hirl efual enurg lxi fvulxivz ...
26990,JHN 21:24\n\n###\n\n,Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual ...


In [133]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py tools fine_tunes.prepare_data -f data/train_exp1.jsonl -q

Analyzing...

- Your file contains 3101 prompt-completion pairs
- More than a third of your `prompt` column/key is uppercase. Uppercase prompts tends to perform worse than a mixture of case encountered in normal language. We recommend to lower case the data if that makes sense in your domain. See https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more details
- All prompts end with suffix `\n\n###\n\n`
- All completions end with suffix `\n###`

Based on the analysis we will perform the following actions:
- [Recommended] Lowercase all your data in column/key `prompt` [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `data/train_exp1_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "data/train_exp1_prepared.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\n\n###\n\n` for the model to start gen

In [134]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.create -t "data/train_exp1.jsonl"

Upload progress: 100%|███████████████████████| 536k/536k [00:00<00:00, 452Mit/s]
Uploaded file from data/train_exp1.jsonl: file-h1HkWU2AQDJVDrUQlN0VRMBJ
Created fine-tune: ft-anCZms8Mp4DC73F3JcDNlVpa
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-29 13:56:53] Created fine-tune: ft-anCZms8Mp4DC73F3JcDNlVpa
^C


In [136]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.create -t "data/train_exp1_prepared.jsonl" -m "davinci"

Upload progress: 100%|███████████████████████| 536k/536k [00:00<00:00, 418Mit/s]
Uploaded file from data/train_exp1_prepared.jsonl: file-oS3PylDXb8l5H5T2HGNo5c9T
Created fine-tune: ft-QVYzWTCt113IwLDQENP4gXYc
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-29 13:58:19] Created fine-tune: ft-QVYzWTCt113IwLDQENP4gXYc

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-QVYzWTCt113IwLDQENP4gXYc



In [156]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.follow -i ft-QVYzWTCt113IwLDQENP4gXYc

[2023-03-29 13:58:19] Created fine-tune: ft-QVYzWTCt113IwLDQENP4gXYc
[2023-03-29 14:06:14] Fine-tune costs $26.20
[2023-03-29 14:06:15] Fine-tune enqueued
[2023-03-29 14:39:14] Fine-tune is in the queue. Queue number: 31
[2023-03-29 14:39:15] Fine-tune is in the queue. Queue number: 30
[2023-03-29 14:39:39] Fine-tune is in the queue. Queue number: 29
[2023-03-29 14:41:17] Fine-tune is in the queue. Queue number: 28
[2023-03-29 14:41:43] Fine-tune is in the queue. Queue number: 27
[2023-03-29 14:47:33] Fine-tune is in the queue. Queue number: 26
[2023-03-29 14:52:16] Fine-tune is in the queue. Queue number: 25
[2023-03-29 14:55:38] Fine-tune is in the queue. Queue number: 24
[2023-03-29 14:56:02] Fine-tune is in the queue. Queue number: 23
[2023-03-29 14:57:08] Fine-tune is in the queue. Queue number: 22
[2023-03-29 14:57:56] Fine-tune is in the queue. Queue number: 21
[2023-03-29 14:59:26] Fine-tune is in the queue. Queue number: 20
[2023-03-29 14:59:30] Fine-tune is in the queue. Queu

In [167]:
import openai
from lib.cipher import substitution_cipher
from nltk.translate.bleu_score import sentence_bleu

openai.api_key = os.getenv("OPENAI_API_KEY")

def question(text, model='davinci'):
    response = openai.Completion.create(
        model=model,
        prompt=text + "\n\n###\n\n",
        temperature=0.5,  # 0.1 is very little randomness/creativity, 2 is very
        max_tokens=100,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n###"]
    )
    
    
    return response.get('choices',[])[0].get('text','').strip()

def translate(text):
    response = openai.Completion.create(
        model="davinci:ft-personal-2023-03-29-23-17-46",
        prompt=text + "\n\n###\n\n",
        temperature=0.5,  # 0.1 is very little randomness/creativity, 2 is very
        max_tokens=100,
        n=1,
        frequency_penalty=0,
        presence_penalty=0,
        #stop=["\n###"]
    )
    result = []
    for choice in response.get('choices',[{}]):
        decoded = substitution_cipher(choice.get('text').strip(), encode=False).strip()
        result.append({
            'translation': choice.get('text').strip(),
            'decoded_translation': decoded,
            #'bleu_score': sentence_bleu([text.split()], decoded.split())
        })
    
    return result

In [168]:
question("What is the verse JHN 3:16 in the Bible say?")

'The verse is:\n\nFor God so loved the world, that he gave his only begotten Son, that whosoever believeth in him should not perish, but have everlasting life.\n\nJohn 3:16 (KJV)\n\n\n\nThe verse is:\n\nFor God so loved the world that he gave his one and only Son, that whoever believes in him shall not perish but have eternal life.\n\nJohn 3:16 (NIV)'

In [165]:
train[train['0'] == 'JHN 3:16'][['0','engBBE','birrig']].sample()

,0,engBBE,birrig
26204,JHN 3:16,For God had such love for the world that he ga...,Suv Guw xew zakx pumi suv lxi huvpw lxel xi ge...


In [166]:
translate("JHN 3:16")

[{'translation': 'Erw lxij hivi holx xon fisuvi xon, holx lxinzipmiz lxij hivi uqirpj lu lxi qpeki hxivi lxij xew fiir qal lu wielx.',
  'decoded_translation': 'And they were with him before him, with themselves they were openly to the place where they had been put to death.'}]

## Feedback
In theory we thought that it could map just from the reference b/c it shows that on devinci it knows the verse.  So in theory it should have all the versions of that text, all the sermons and all the commentaries and comments about it embedded into it.  But when asked for that verse after training it then it doesn't know it so it appears to not be packaging all those findings with it.